In [3]:
import pandas as pd
# Load datasets
train = pd.read_csv('/Users/ad53533/Desktop/Applied ML/Project/train.csv')
missing_sii_rows = train['sii'].isnull().sum()
print(missing_sii_rows)

1224


In [4]:
len(train)

3960

In [11]:
import pandas as pd
from sklearn.impute import KNNImputer

# Step 1: Drop 'id' feature and assign 'missing' to missing categorical values
train = train.drop(columns=['id'], errors='ignore')  # Drop 'id' if it exists
categorical_features = train.select_dtypes(include=['object', 'category']).columns

# Fill missing values in categorical features with 'missing'
train[categorical_features] = train[categorical_features].fillna('missing')

# Dummy encode the categorical variables
train = pd.get_dummies(train, columns=categorical_features, drop_first=True)

# Step 2: Drop rows with missing 'sii' values
# Get rows with missing 'sii' values
missing_sii_rows_df = train[train['sii'].isnull()]
train = train.dropna(subset=['sii'], axis=0)

# Step 3: Group by 'sii' values and impute each group individually
groups = train.groupby('sii')
imputed_groups = []

for sii_value, group in groups:
    # Drop 'sii' column temporarily as it doesn't need imputation
    sii_column = group['sii']
    group = group.drop(columns=['sii'])
    
    # Apply KNN Imputer
    imputer = KNNImputer(n_neighbors=5)  # Adjust `n_neighbors` as needed
    imputed_data = imputer.fit_transform(group)
    
    # Recreate DataFrame with imputed data
    imputed_group = pd.DataFrame(imputed_data, columns=group.columns, index=group.index)
    imputed_group['sii'] = sii_column  # Add 'sii' column back
    imputed_groups.append(imputed_group)

# Step 4: Concatenate all groups back into a single DataFrame
imputed_train = pd.concat(imputed_groups)

# The final DataFrame `imputed_train` contains:
# - Imputed numerical data
# - Dummy-encoded categorical data
# - 'sii' values retained


In [40]:
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics.pairwise import euclidean_distances

# Assuming `imputed_train` is your dataset and `new_vector` is the incoming vector with missing values

# Step 1: Ensure the new vector is a DataFrame with the same columns as the dataset
new_vector_df = pd.DataFrame(missing_sii_rows_df, columns=imputed_train.columns)

# Step 2: Handle missing values in the new vector using the same KNNImputer
imputer = KNNImputer(n_neighbors=3)  # Use the same imputer as before
combined_data = pd.concat([imputed_train, new_vector_df], ignore_index=True)
imputed_combined = imputer.fit_transform(combined_data)

test_indices = range(1000)
count = 0
for indices in test_indices:
    

    # Extract the imputed new vector
    imputed_new_vector = imputed_combined[-indices]  # The last row corresponds to the new vector
    
    # Step 3: Compute distances
    distances = euclidean_distances(imputed_combined[:-1], [imputed_new_vector])  # Exclude the new vector from the dataset
    
    # Step 4: Find the 5 nearest neighbors
    nearest_indices = np.argsort(distances.flatten())[:3]
    nearest_distances = distances.flatten()[nearest_indices]
    
    # # Print results
    # print("Indices of the 5 nearest neighbors:", nearest_indices)
    # print("Distances to the 5 nearest neighbors:", nearest_distances)
    
    # Optionally, you can retrieve the corresponding rows in the original DataFrame
    nearest_neighbors = imputed_train.iloc[nearest_indices]
    nearest_sii = nearest_neighbors['sii']
    sii_values = nearest_neighbors['sii'].unique()
    if len(sii_values) == 1:  # If all 'sii' values are the same
        # print(f"All nearest neighbors have the same 'sii' value: {sii_values[0]}")
        count = count + 1
    # print("Nearest Neighbors DataFrame:")
    # print(nearest_neighbors)
print(count)


297


In [31]:
range(10)

range(0, 10)